In [1]:
import cv2
import matplotlib.pyplot as plt
import mediapipe as mp
import util
import pyautogui
from pynput.mouse import Button, Controller
import random

mouse = Controller()

In [2]:
screen_width, screen_height = pyautogui.size()

mpHands = mp.solutions.hands
hands = mpHands.Hands( static_image_mode = False,
                     model_complexity = 1,
                     min_detection_confidence = 0.7,
                     min_tracking_confidence = 0.7,
                     max_num_hands = 1)

In [3]:
def find_finger_tip(process):
    if process.multi_hand_landmarks:
        hand_landmarks = process.multi_hand_landmarks[0]
        return hand_landmarks.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP]
    return None

In [4]:
def move_mouse(index_finger_tip):
    if index_finger_tip is not None:
        x = int(index_finger_tip.x * screen_width)
        y = int(index_finger_tip.y * screen_height)
        pyautogui.moveTo(x,y)

In [5]:
def is_left_click(landmarks_list, thumb_index_dist):
    return(util.get_angle(landmarks_list[5],landmarks_list[6],landmarks_list[8]) < 50 and
          util.get_angle(landmarks_list[9],landmarks_list[10],landmarks_list[12]) > 87 and
           thumb_index_dist > 40)
    
def is_right_click(landmarks_list, thumb_index_dist):
    return(util.get_angle(landmarks_list[5],landmarks_list[6],landmarks_list[8]) > 87 and
          util.get_angle(landmarks_list[9],landmarks_list[10],landmarks_list[12]) < 50 and
           thumb_index_dist > 40)
    
def is_double_click(landmarks_list, thumb_index_dist):
    return(util.get_angle(landmarks_list[5],landmarks_list[6],landmarks_list[8]) < 50 and
          util.get_angle(landmarks_list[9],landmarks_list[10],landmarks_list[12]) < 50 and
           thumb_index_dist > 40)
    
def is_screenshot(landmarks_list, thumb_index_dist):
    return(util.get_angle(landmarks_list[5],landmarks_list[6],landmarks_list[8]) < 50 and
          util.get_angle(landmarks_list[9],landmarks_list[10],landmarks_list[12]) < 50 and
           thumb_index_dist < 40)

In [6]:
def detect_gestures(frame,landmarks_list,process):
    if len(landmarks_list) >= 21:
        index_finger_tip = find_finger_tip(process)
        thumb_index_distance = util.get_dist([landmarks_list[4], landmarks_list[5]])
        # angel = util.get_angle(landmarks_list[5],landmarks_list[6],landmarks_list[8])
#cursour movement code
        if thumb_index_distance < 40 and util.get_angle(landmarks_list[5],landmarks_list[6],landmarks_list[8]) > 90:
            move_mouse(index_finger_tip)
            # print(thumb_index_distance)

#left click code
        elif is_left_click(landmarks_list,thumb_index_distance):
            mouse.press(Button.left)
            mouse.release(Button.left)
            cv2.putText(frame,"Left Click",(50,50), cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)

#right click code
        elif is_right_click(landmarks_list,thumb_index_distance):
            mouse.press(Button.right)
            mouse.release(Button.right)
            cv2.putText(frame,"Right Click",(50,50), cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)

#double click code
        elif is_double_click(landmarks_list,thumb_index_distance):
            pyautogui.doubleClick()
            cv2.putText(frame,"Double Click",(50,50), cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,0),2)
            
#Screenshot code
        elif is_screenshot(landmarks_list,thumb_index_distance):
            im1 = pyautogui.screenshot()
            label = random.randint(1, 1000)
            im1.save(f'my_screenshot_{label}.png')
            cv2.putText(frame, "Screenshot Taken", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

In [10]:
def main():
    # mp_hands = mp.solutions.hands
    # hands = mp_hands.Hands(max_num_hands = 2)
    
    #video capturing
    cap = cv2.VideoCapture(0)
    draw = mp.solutions.drawing_utils

    try:
        while True:
            ret, frame = cap.read()  # Capture frame-by-frame
            if not ret:
                break  # If frame not captured, exit loop
            frame = cv2.flip(frame,1)
            rgb_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
            process = hands.process(rgb_frame)
            landmarks_list = []
            if process.multi_hand_landmarks:
                hand_landmarks = process.multi_hand_landmarks[0]
                draw.draw_landmarks(frame,hand_landmarks, mpHands.HAND_CONNECTIONS)

                for lm in hand_landmarks.landmark:
                    landmarks_list.append((lm.x , lm.y))

            detect_gestures(frame,landmarks_list,process)
                # print(landmarks_list)
            cv2.imshow('Real-Time Video', frame)  # Display the frame
        
            # Press 'q' to quit
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    finally:
        cap.release()  # Release the webcam
        cv2.destroyAllWindows()  # Close all OpenCV windows

if __name__ == "__main__":
    main()

In [13]:
def main():
    cap = cv2.VideoCapture(0)
    draw = mp.solutions.drawing_utils
    try:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.flip(frame,1)
            frame_rgb = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
            

SyntaxError: incomplete input (2318177465.py, line 11)